In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer

import joblib


In [ ]:
#log-transform
def log_transform(X, columns):
    X = X.copy()
    for col in columns:
        X[col] = np.log(X[col])
    return X

In [ ]:
def update_column_names(df):
    df = df.rename(columns={
        'Temperature': 'Temperatura',
        'Humidity': 'Umidade',
        'Wind Speed': 'Velocidade_do_Vento',
        'Cloud Cover': 'Cobertura_de_Nuvens',
        'Pressure': 'Pressao',
        'Location': 'Localizacao',
        'Rain Tomorrow': 'Chovera_Amanha',
        'Precipitation': 'Precipitacao',
        'Date':'Data',
    })
    return df

In [ ]:
def get_season(date):
        month = date.month
        day = date.day
        if ((month >= 3 and day >= 20) and (month <= 6 and day < 21)):
                return 'Primavera'
        elif (month >= 6 and ( month <= 9 and day < 23)):
                return 'Verão'
        elif (month >= 9 and (month <= 12 and day < 21)):
                return 'Outono'
        else:
                return 'Inverno'

In [ ]:
# Mapeamento completo de cidades para estados
cidade_para_estado = {
    'Los Angeles': 'California', 
    'Chicago': 'Illinois', 
    'Houston': 'Texas',
    'Phoenix': 'Arizona',
    'Philadelphia': 'Pennsylvania',
    'San Antonio': 'Texas', 
    'San Diego': 'California',
    'Dallas': 'Texas',
    'San Jose': 'California', 
    'Austin': 'Texas', 
    'Jacksonville': 'Florida', 
    'Fort Worth': 'Texas',
    'Columbus': 'Ohio', 
    'Indianapolis': 'Indiana', 
    'Charlotte': 'North Carolina', 
    'San Francisco': 'California',
    'Seattle': 'Washington', 
    'Denver': 'Colorado', 
    'New York': 'New York', 
    'Washington D.C.': 
    'District of Columbia'
}
estado_para_regiao = {
    'California': 'Oeste',
    'Illinois': 'Meio-Oeste',
    'Texas': 'Sul',
    'Arizona': 'Oeste',
    'Pennsylvania': 'Nordeste',
    'Florida': 'Sul',
    'Ohio': 'Meio-Oeste',
    'Indiana': 'Meio-Oeste',
    'North Carolina': 'Sul',
    'Washington': 'Oeste',
    'Colorado': 'Oeste',
    'New York': 'Nordeste',
    'District of Columbia': 'Sul'
}
def cidade_para_regiao(cidade: pd.Series) -> str:
    cidade = cidade.title()
    estado = cidade_para_estado.get(cidade)
    return estado_para_regiao.get(estado) if estado else None

In [ ]:
numeric_features_pt = ['Temperatura', 'Umidade', 'Velocidade_do_Vento', 'Cobertura_de_Nuvens', 'Pressao']
categorical_features_pt = ['Estacao', 'Regiao']

def pre_processar_df(df:pd.DataFrame, numeric_columns):
    # Transformando os dados numéricos para o log
    df = log_transform(df, numeric_columns)

    df.sort_values(by='Data', inplace=True, ascending=True)

    # Cria a coluna Estacao
    df['Data'] = pd.to_datetime(df['Data'])
    
    df["Estacao"] = df["Data"].apply(get_season)
    
    # Cria a coluna Regiao
    df['Regiao'] = df['Localizacao'].apply(cidade_para_regiao)

    # Remove colunas originais que não serão mais usadas
    df = df.drop(['Data', 'Localizacao'], axis=1)
    return df

# Para usar no pipeline:
preprocess_func = FunctionTransformer(pre_processar_df, kw_args={'numeric_columns': numeric_features_pt}, validate=False)


preprocess_before_transforms_pipeline = Pipeline([
    ('preprocess_func', preprocess_func)
])

# Transformadores

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Preprocessamento
# Cria o pré-processador

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features_pt),
    ('cat', categorical_transformer, categorical_features_pt)
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocess_func', preprocess_func),
    ('preprocessor', preprocessor)
])

In [ ]:
df = pd.read_csv('usa_rain_prediction_dataset_2024_2025.csv')
df = update_column_names(df)

X = df.drop(columns=['Chovera_Amanha'])
y = df['Chovera_Amanha'].values

_,X_test,_,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = pipeline.transform(X_test)

# Carregando o modelo
modelo = joblib.load('modelo_rain_prediction.pkl')
# Fazendo previsões
y_pred = modelo.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Sensibilidade: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# plotando as métricas
def plot_metrics(y_test, y_pred):
    metrics = {
        'Acurácia': accuracy_score(y_test, y_pred),
        'Precisão': precision_score(y_test, y_pred),
        'Sensibilidade': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred)
    }

    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(metrics.keys()), y=list(metrics.values()))
    plt.title('Métricas de Avaliação do Modelo')
    plt.ylabel('Valor')
    plt.xlabel('Métricas')
    plt.ylim(0, 1)
    plt.show()

